## Extract, Transform, and Load Airbnb Data into SQLite Database

In [1]:
import pandas as pd
import sqlite3
from sqlite3 import Error
db_file = '..\data\db.sqlite'

In [2]:
# Read csv file into pandas dataframe
airbnb_df = pd.read_csv('..\data\AB_NYC_2019.csv')

In [3]:
# Inspect dataframe
airbnb_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [4]:
# Get info on dataframe
airbnb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [5]:
# Convert dataframe to json and save to file
airbnb_df.to_json('..\data\AB_NYC_2019.json')

## Create a Connection to an SQLite Database & Tables within it

In [6]:
# Define a function to create a database connection
def create_connection(db_file):
    """ create a database connection to a SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
        
    return conn

In [7]:
# Define a function to create a table
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [8]:
# Reference df columns for creating table
airbnb_df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [11]:
# Define a function to create a project
def main():
    database = db_file
    
    sql_create_airbnb_table = """ CREATE TABLE IF NOT EXISTS airbnb (
                                        id integer PRIMARY KEY NOT NULL,
                                        name text NOT NULL,
                                        host_id integer NOT NULL,
                                        host_name text NOT NULL,
                                        neighbourhood_group text NOT NULL,
                                        neighbourhood text NOT NULL,
                                        latitude real NOT NULL,
                                        longitude real NOT NULL,
                                        room_type text NOT NULL,
                                        price integer NOT NULL,
                                        minimum_nights integer NOT NULL,
                                        number_of_reviews integer NOT NULL,
                                        last_review text NOT NULL,
                                        reviews_per_month real NOT NULL,
                                        calculated_host_listings_count integer NOT NULL,
                                        availability_365 integer NOT NULL
                                    ); """
    # Create a database connection
    conn = create_connection(database)

    # Create table
    if conn is not None:
        # Create airbnb table
        create_table(conn, sql_create_airbnb_table)
    else:
        print("Error! cannot create the database connection.")
                                        

In [12]:
if __name__ == '__main__':
    main()

## Add records to sqlite database

### Note: This involves create_connection(), create_record(), and main() functions.
### - create_connection() creates a database connection to the SQLite database specified by db_file. (it already exists)
### - create_record() inserts a new record into the airbnb table.
### - main() calls the create_connection() function to create a database connection and calls the create_record() function to insert a new record.


In [9]:
# Define function to create insert a new record into the airbnb table
def create_record(conn, record):
    sql = ''' INSERT INTO airbnb(id,name,host_id,host_name,neighbourhood_group,
                neighbourhood,latitude,longitude,room_type,price,minimun_nights,number_of_reviews,
                    last_review,reviews_per_month,calculated_host_listings_count,availability_365)
                        VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, record)
    conn.commit()
    return cur.lastrowid

In [10]:
# Define main() function to call create_connection() and create_record() functions
def main():
    # Define database
    database = db_file
    
    # Create connection to database
    conn = create_connection(database)
    
    for row in airbnb_df.itertuples():
        with conn:
            record = (row.id, row.name, row.host_id, row.host_name, row.neighbourhood_group, row.neighbourhood,
                      row.latitude, row.longitude, row.room_type, row.price, row.minimum_nights, row.number_of_reviews,
                      row.last_review, row.reviews_per_month, row.calculated_host_listings_count, row.availability_365)
            create_record(conn, record)

In [14]:
# Call main() function
if __name__ == '__main__':
    main()

OperationalError: no such table: airbnb_table

In [ ]:
# Create function to drop table if exists
def drop_table(conn, table_name):
    drop_table_sql = f"DROP TABLE IF EXISTS {table_name};"

# Call drop_table function
drop_table(conn, 'airbnb_table')

# Close connection
conn.close()